In [ ]:
#| default_exp utils/r

You can install the package with `pip install rpy2 rpy2-arrow`

In [ ]:
#| export
import rpy2
import rpy2_arrow.arrow as pyra
from rpy2.robjects.packages import importr
import polars as pl

def py2rpy_polars():
    #| code-summary: helper functions to convert between `python` and `R` dataframes
    base = importr('base')

    conv_pl = rpy2.robjects.conversion.Converter(
        'Pandas to pyarrow',
        template=pyra.converter)

    @conv_pl.py2rpy.register(pl.DataFrame)
    def py2rpy_pandas(dataf: pl.DataFrame):
        pa_tbl = dataf.to_arrow()
        return base.as_data_frame(pa_tbl)
        # return pyra.converter.py2rpy(pa_tbl) # NOTE: not working for ggplot2

    conv_pl = rpy2.ipython.rmagic.converter + conv_pl
    return conv_pl

In [ ]:
# test
%load_ext rpy2.ipython
conv_pl = py2rpy_polars()
df = pl.DataFrame({
    'a': [1, 2, 3],
    'b': [4, 5, 6],
    'c': ['a', 'b', 'c']
})
%R -i df -c conv_pl
%R print(df)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
  a b c
1 1 4 a
2 2 5 b
3 3 6 c


,a,b,c
1,1,4,a
2,2,5,b
3,3,6,c
